## install and import libraries

In [ ]:
import pandas as pd 
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn import neighbors
import time
from sklearn.decomposition import TruncatedSVD
import ipywidgets
from ipywidgets import interact
from ipywidgets import interact_manual

## read data

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommender Systems/fyp/all_transaction_data.csv')

data.head()

<ipython-input-46-0f2ae9eb839d>:1: DtypeWarning: Columns (6,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommender Systems/fyp/all_transaction_data.csv')


,Address/Project Name,Date of Sale,Sale Price,Private/Public,Type,Floor_min,Floor_max,Area (Sqm),Remaining Lease (2022),Flat_model
0,PEOPLE'S PARK COMPLEX,2022-09,1060000.0,Private,Apartment,11,15,104.0,45 years,NaN
1,THE SAIL @ MARINA BAY,2022-09,1300000.0,Private,Apartment,61,65,63.0,79 years,NaN
2,MARINA ONE RESIDENCES,2022-08,1620000.0,Private,Apartment,26,30,65.0,88 years,NaN
3,MARINA ONE RESIDENCES,2022-08,3744171.0,Private,Apartment,26,30,141.0,88 years,NaN
4,ONE SHENTON,2022-08,1100000.0,Private,Apartment,11,15,54.0,82 years,NaN


## clean data

In [ ]:
# check
data.loc[0] 

,Address/Project Name,Date of Sale,Sale Price,Private/Public,Type,Floor_min,Floor_max,Area (Sqm),Remaining Lease (2022),Flat_model
0,PEOPLE'S PARK COMPLEX,2022-09,1060000.0,Private,Apartment,11,15,104.0,45 years,NaN
0,406 ANG MO KIO AVENUE 10,2017-01,232000.0,Public,2 ROOM,10,12,44.0,61 years 04 months,Improved


In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
# check
data['Floor_min'].unique() 

array(['11', '61', '26', '36', '21', '01', '56', '16', '46', '06', '31',
       '41', '51', '66', nan, 'B1', '71', '10', '04', '07', '13', '19',
       '22', '34', '28', '37', '49', '25', '40', 7, 4, 16, 10, 19, 13, 31,
       22, 28, 49, 46, 40, 1, 34, 25, 37, 43], dtype=object)

In [ ]:
# check
data['Floor_max'].unique() 

array(['15', '65', '30', '40', '25', '05', '60', '20', '50', '10', '35',
       '45', '55', '70', nan, 'B5', '75', '12', '03', '06', '09', '21',
       '24', '18', '36', '39', '51', '27', '42', '33', 9, 6, 18, 12, 21,
       15, 33, 24, 30, 51, 48, 42, 3, 36, 27, 39, 45], dtype=object)

In [ ]:
data['Floor_min'] = data['Floor_min'].astype(str)

In [ ]:
data['Floor_max'] = data['Floor_max'].astype(str)

In [ ]:
# check
data.isnull().sum()

Address/Project Name           0
Date of Sale                   0
Sale Price                     0
Private/Public                 0
Type                           0
Floor_min                      0
Floor_max                      0
Area (Sqm)                     0
Remaining Lease (2022)         0
Flat_model                128373
dtype: int64

In [ ]:
data['Flat_model'].fillna('Unknown', inplace = True)

In [ ]:
# check
data.duplicated().any()

True

In [ ]:
data.drop_duplicates(keep="first", inplace=True)

## encode data

In [ ]:
date_df = pd.get_dummies(data['Date of Sale'])

date_df.head()

,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,...,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
priv_df = pd.get_dummies(data['Private/Public'])

priv_df.head()

,Private,Public
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [ ]:
type_df = pd.get_dummies(data['Type'])

type_df.head()

,1 ROOM,2 ROOM,3 ROOM,4 ROOM,5 ROOM,Apartment,Condominium,Detached,EXECUTIVE,Executive Condominium,MULTI-GENERATION,Semi-detached,Strata Detached,Strata Semi-detached,Strata Terrace,Terrace
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
floor_min_df = pd.get_dummies(data['Floor_min'])

floor_min_df.head()

,01,04,06,07,1,10,11,13,16,19,...,46,49,51,56,61,66,7,71,B1,nan
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
floor_max_df = pd.get_dummies(data['Floor_max'])

floor_max_df.head()

,03,05,06,09,10,12,15,18,20,21,...,51,55,6,60,65,70,75,9,B5,nan
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
lease_df = pd.get_dummies(data['Remaining Lease (2022)'])

lease_df.head()

,-1 years,105 years,12 years,15 years,24 years,26 years,27 years,29 years,31 years,34 years,...,996 years,9970 years,9972 years,9981 years,999935 years,999936 years,999937 years,999940 years,999970 years,Freehold
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
flat_df = pd.get_dummies(data['Flat_model'])

flat_df.head()

,2-room,3Gen,Adjoined flat,Apartment,DBSS,Improved,Improved-Maisonette,Maisonette,Model A,Model A-Maisonette,...,New Generation,Premium Apartment,Premium Apartment Loft,Premium Maisonette,Simplified,Standard,Terrace,Type S1,Type S2,Unknown
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## dimensionalty reduction

In [ ]:
features = pd.concat([data['Address/Project Name'], date_df, data['Sale Price'], priv_df, type_df, floor_min_df, floor_max_df, lease_df, data['Area (Sqm)'], flat_df], axis=1)
features.set_index('Address/Project Name', inplace=True)

features.head()

,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,...,New Generation,Premium Apartment,Premium Apartment Loft,Premium Maisonette,Simplified,Standard,Terrace,Type S1,Type S2,Unknown
Address/Project Name,,,,,,,,,,,,,,,,,,,,,
PEOPLE'S PARK COMPLEX,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
THE SAIL @ MARINA BAY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
MARINA ONE RESIDENCES,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
MARINA ONE RESIDENCES,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ONE SHENTON,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
svd = TruncatedSVD(n_components=9, random_state=42)

t0 = time.time()
features_scaled = svd.fit_transform(features)
t1 = time.time()
time_taken_1 = t1 - t0
print(time_taken_1)

26.67888116836548


In [ ]:
features_scaled.shape

(259256, 9)

## recommendation system 

### implement model

In [ ]:
model = neighbors.NearestNeighbors(n_neighbors=4, algorithm='kd_tree', metric='euclidean')

t0 = time.time()
model.fit(features_scaled)
t1 = time.time()
time_taken_1 = t1 - t0
print(time_taken_1)
print('\n')

dist, idlist = model.kneighbors(features_scaled)
t2 = time.time()
time_taken_2 = t2 - t1
print(time_taken_2)

0.40708017349243164


2.0117759704589844


In [ ]:
index = 0
length = len(dist)

new_dist = []
new_idlist = []

while index < length:
  dist_item = dist[index][1:]
  id_item = idlist[index][1:]
  new_dist.append(dist_item)
  new_idlist.append(id_item)
  index = index + 1

### names only

In [ ]:
@interact
def PlaceRecommender(place_name = list(data['Address/Project Name'].value_counts().index)):
    place_list_name = []
    place_id = data[data['Address/Project Name'] == place_name].index
    place_id = place_id[0]
    for newid in new_idlist[place_id]:
      place_list_name.append(data.loc[newid,'Address/Project Name'])
    return place_list_name

interactive(children=(Dropdown(description='place_name', options=('LANDED HOUSING DEVELOPMENT', 'TREASURE AT T…

In [ ]:
PlaceRecommender('RIVERFRONT RESIDENCES')

['STIRLING RESIDENCES', 'QUEENS PEAK', 'THE FLORENCE RESIDENCES']

In [ ]:
PlaceRecommender('662 YISHUN AVENUE 4')

['34 BEDOK SOUTH AVENUE 2',
 '130 ANG MO KIO AVENUE 3',
 '682B WOODLANDS DRIVE 62']

In [ ]:
new_df = pd.DataFrame(PlaceRecommender('662 YISHUN AVENUE 4'))

new_df

,0
0,34 BEDOK SOUTH AVENUE 2
1,130 ANG MO KIO AVENUE 3
2,682B WOODLANDS DRIVE 62


### all fields

In [ ]:
@interact
def PlaceRecommender2(place_name = list(data['Address/Project Name'].value_counts().index)):
    place_list_name = []
    place_id = data[data['Address/Project Name'] == place_name].index
    place_id = place_id[0]
    for newid in new_idlist[place_id]:
      place_list_name.append(data.loc[newid])
    return place_list_name

interactive(children=(Dropdown(description='place_name', options=('LANDED HOUSING DEVELOPMENT', 'TREASURE AT T…

In [ ]:
data.loc[0]

Address/Project Name      PEOPLE'S PARK COMPLEX
Date of Sale                            2022-09
Sale Price                            1060000.0
Private/Public                          Private
Type                                  Apartment
Floor_min                                    11
Floor_max                                    15
Area (Sqm)                                104.0
Remaining Lease (2022)                 45 years
Flat_model                              Unknown
Name: 0, dtype: object

In [ ]:
new_df = pd.DataFrame(PlaceRecommender2('662 YISHUN AVENUE 4'))

new_df

,Address/Project Name,Date of Sale,Sale Price,Private/Public,Type,Floor_min,Floor_max,Area (Sqm),Remaining Lease (2022),Flat_model
138248,34 BEDOK SOUTH AVENUE 2,2017-07,391000.0,Public,4 ROOM,13,15,90.0,59 years 03 months,Improved
232341,130 ANG MO KIO AVENUE 3,2021-08,350000.0,Public,3 ROOM,13,15,73.0,56 years 05 months,New Generation
201122,682B WOODLANDS DRIVE 62,2020-06,430000.0,Public,5 ROOM,1,3,120.0,78 years 09 months,Premium Apartment


## export model

In [ ]:
test_list = [
"THE SAIL @ MARINA BAY",
"GOLDEN MILE COMPLEX",
"TANQUEELAN SUITES",
"MARINA ONE RESIDENCES",
"HERITAGE PLACE",
"ONE SHENTON",
"MARINA BAY SUITES",
"V ON SHENTON",
"MARINA BAY RESIDENCES",
"THE CLIFT",
"EMERALD GARDEN",
"STURDEE RESIDENCES",
"ROBINSON SUITES",
"THE RIVERSIDE PIAZZA",
"EDEN RESIDENCES CAPITOL",
"HIGH STREET CENTRE",
"CITY GATE",
"DUO RESIDENCES",
"TEXTILE CENTRE",
"THE PLAZA",
"FORTE SUITES",
"CITYLIGHTS",
"CONCOURSE SKYLINE",
"ST FRANCIS COURT",
"AMARYLLIS VILLE",
"57 HAVELOCK ROAD",
"69 REDHILL CLOSE",
"225 ANG MO KIO AVENUE 1",
"443 ANG MO KIO AVENUE 10",
"170 ANG MO KIO AVENUE 4",
"27A JALAN MEMBINA",
"219A BEDOK CENTRAL",
"173 YISHUN AVENUE 7",
"412 YISHUN RING ROAD",
"425 BEDOK NORTH ROAD",
"206 YISHUN STREET 21",
"435 BUKIT PANJANG RING ROAD",
"201 BEDOK NORTH STREET 1",
"8 TOH YI DRIVE",
"187 BUKIT BATOK WEST AVENUE 6",
"257 BUKIT BATOK EAST AVENUE 4",
"288 CHOA CHU KANG AVENUE 3",
"476B CHOA CHU KANG AVENUE 5",
"423 CLEMENTI AVENUE 1",
"329 CLEMENTI AVENUE 2",
"122 BUKIT BATOK CENTRAL",
"262 TAMPINES STREET 21",
"60 TELOK BLANGAH HEIGHTS",
"42 TELOK BLANGAH RISE",
"233 SIMEI STREET 4"
]

In [ ]:
class SomeThing:
    def __init__(self, place, rec1, rec2, rec3):
        self.place, self.rec1, self.rec2, self.rec3 = place, rec1, rec2, rec3

In [ ]:
things = []

for place in test_list:
  rec = PlaceRecommender(place)
  things.append(SomeThing(place, rec[0], rec[1], rec[2]))

test_df = pd.DataFrame([t.__dict__ for t in things ])

In [ ]:
# check
test_df

,place,rec1,rec2,rec3
0,THE SAIL @ MARINA BAY,MARINA ONE RESIDENCES,TWIN WATERFALLS,ESPARINA RESIDENCES
1,GOLDEN MILE COMPLEX,FLO RESIDENCE,AFFINITY AT SERANGOON,HELIOS RESIDENCES
2,TANQUEELAN SUITES,DUO RESIDENCES,OLEANAS RESIDENCE,JADESCAPE
3,MARINA ONE RESIDENCES,MARINA ONE RESIDENCES,PICCADILLY GRAND,AVENUE SOUTH RESIDENCE
4,HERITAGE PLACE,YEW MEI GREEN,YEW MEI GREEN,ESPARINA RESIDENCES
5,ONE SHENTON,AMBER RESIDENCES,THE CAPE,PEBBLE BAY
6,MARINA BAY SUITES,CASTLE LOFT,PARC BOTANNIA,THE SCALA
7,V ON SHENTON,FLORIDIAN,LAKE GRANDE,TIARA
8,MARINA BAY RESIDENCES,THE MORNINGSIDE,THE HILLFORD,SIGNATURE PARK
9,THE CLIFT,LIV @ MB,SEAHILL,THE GARDENS AT BISHAN


In [ ]:
test_df.to_csv('recommended_places.csv', index = False)

In [ ]:
# Convert dataframe to JSON
json_data = test_df.to_json('recommendation_system.json', orient='index')

# Print the JSON string
print(json_data)

None
